In [116]:
import numpy as np
import pandas as pd

## Duration of Leave

In [119]:
class EmergencyData:
    '''create a data frame and compute duration of leave with some statistics'''
    def __init__(self, emergency_data_file, visit_date, discharge_date):
        '''create '''
        self.data = self._creating_data_frame(emergency_data_file)
        self.duration_of_leave = self._computing_duration_of_leave(self.data, visit_date, discharge_date)
        self.duration_of_leave_statistics = self._computing_duration_of_leave_statistics(self.duration_of_leave)
        self.percentage_of_duration_specific_hours = self._percentage_durations_less_than_four_hours(self.duration_of_leave)
        self.percentage_of_duration_within_10_20_30_minutes = self._percentage_duration_within_10_20_30_minutes(self.duration_of_leave)
    
    def _creating_data_frame(self, file):
        '''create data frame'''
        self.df = pd.read_excel(file)
        return self.df
    
    def _computing_duration_of_leave(self, df, col1, col2):
        '''compute duration of leave'''
        self.df['DURATION_OF_LEAVE'] = df[col2] - df[col1]
        return self.df['DURATION_OF_LEAVE']
    
    def _computing_duration_of_leave_statistics(self, series):
        '''compute basic statistics'''
        self.stats = pd.DataFrame(series.describe())
        return self.stats
    
    def _percentage_durations_less_than_four_hours(self, series):
        '''compute percentage of leave duration for patients who left less than or within and more than four hours '''
        less_than_four_hours = round(series[series < '0 days 4:00:00'].shape[0] / series.shape[0] * 100)
        more_than_or_equal_four_hours = round(series[series >= '0 days 4:00:00'].shape[0] / series.shape[0] * 100)
        
        self.df = pd.DataFrame(
                [str(less_than_four_hours)+'%', str(more_than_or_equal_four_hours)+'%'],
                index = ['Less than four hours', 'More than four hours'],
                columns= ['Percentage'],
                    ).transpose()
        return self.df
    
    def _percentage_duration_within_10_20_30_minutes(self, series):
        '''compute percentage of leave duration within specific times'''
        within_10_minutes = round(series[series <= '0 days 00:10:00'].shape[0] / series.shape[0] * 100)
        within_20_minutes = round(series[series <= '0 days 00:20:00'].shape[0] / series.shape[0] * 100)
        within_30_minutes = round(series[series <= '0 days 00:30:00'].shape[0] / series.shape[0] * 100)
        
        self.df = pd.DataFrame(
                [str(within_10_minutes)+'%', str(within_20_minutes)+'%', str(within_30_minutes)+'%'],
                index=['within_10_minutes', 'within_20_minutes', 'within_30_minutes'],
                columns= ["Percentage"],
                    ).transpose()
        return self.df

In [120]:
class AnlyzingFeature:
    """ 
    Analyzing the following features:
    - Consultations
    - Lab Tests
    - Radilogy
    - Pharmach
    
    """
    def __init__(self, data, feature):
        '''create variables which can give us specific value/s'''
        self.single_feature = self._get_feature(data, feature)
        self.series_without_null_vals = self._ignore_null_values(self.single_feature)
        self.vals_as_lst = self._split_text(self.series_without_null_vals)
        self.order_counts_df = self._find_length_text(self.vals_as_lst, feature)
        self.lst_len_orders = self._compute_length(self.vals_as_lst)
        self.lst_dates_times = self._extract_dates_times(self.vals_as_lst)
        self.duration = self._compute_durations(self.lst_dates_times)
        self.unique_no = self._unique_of_length_or_duration_stat(self.lst_len_orders)
        self.duration_stats = self._unique_of_length_or_duration_stat(self.duration)
        
    def _get_feature(self, df, column_name):
        '''get feature from dataframe'''
        series = df.data[column_name]
        return series
    
    def _ignore_null_values(self,series):
        '''ignore null values'''
        mask = series.isna()
        series = series[~mask]
        return series
        
    def _split_text(self,series):
        '''split each order in the series '''
        series_with_lsts = series.apply(lambda x: x.split(';'))
        return series_with_lsts
    
    def _find_length_text(self, series, column_name):
        '''compute number of orders for each patient to find frequent of orders'''
        series_number = series.apply(lambda x: len(x)).value_counts()
        frequent_orders_df = pd.DataFrame(series_number).rename(columns={str(column_name):'Frquent'})
        return frequent_orders_df
        
    def _compute_length(self,series):
        '''find length of each order'''
        len_orders = series.apply(lambda x: [len(n) for n in x])
        return len_orders
    
    def _unique_of_length_or_duration_stat(self, series):
        '''
        find length of text and count of orders as well as date time 
        with and without Reply or End "Invalid Value" and duration statistics
        '''
        total_lst = []
        for lst in series:
            total_lst.extend(lst)
            
        if str(series) != str(self.duration):
            if self.single_feature.name == 'CONSULTATIONS':
                num = 47
            else:
                num = 58
                    
            len_of_text = pd.Series(total_lst).unique()

            len_without_reply = len([n for n in total_lst if n<num])
            len_with_reply = len([n for n in total_lst if n==num])

            per_without_reply = str(round(len([n for n in total_lst if n<num]) / len(total_lst) * 100))+"%"
            per_with_reply = str(round(len([n for n in total_lst if n==num]) / len(total_lst) * 100))+'%'
            
            try:
                without_with_reply_details = pd.DataFrame(
                                    [
                                     (len_of_text[1],len_of_text[0]),
                                     (len_without_reply,len_with_reply), 
                                     (per_without_reply,per_with_reply)
                                    ],
                                    columns=['Without Reply/End"Invalid Value', 'With Reply/End'],
                                    index = [
                                        'Length of Text "Characters"',
                                        'Count of consultations',
                                        'Percentage of consultations'
                                    ]
                                    )
                return without_with_reply_details
            
            except:
                
                without_with_reply_details = pd.DataFrame(
                                    [
                                     len_of_text,
                                     (len_without_reply,len_with_reply), 
                                     (per_without_reply,per_with_reply)
                                    ],
                                    columns=['With Reply/End','Without Reply/End "Invalid Value"'],
                                    index = [
                                        'Length of Text "Characters"',
                                        'Count of consultations',
                                        'Percentage of consultations'
                                    ]
                                    )
                return without_with_reply_details
        else:
            return pd.DataFrame(
                                pd.Series(total_lst).describe(), 
                                columns=['Statistics']
                                )
    
    def _extract_dates_times(self, series):
        '''extract dates and times for all orders'''
        if self.single_feature.name == 'CONSULTATIONS':
            series_date_time = series.apply(lambda x: [ 
                                            x[i][8:25] +'  '+ x[i][31:] \
                                            for i in range(len(x)) 
                                            ])
            return series_date_time
        
        else:
            series_date_time = series.apply(lambda x: 
                                            ' '.join(x).split('START: ,')[1:]
                                           )
            return series_date_time
    
    def _compute_durations(self, series):
        '''compute durations by subtract Reply or End date time from Request or Start date time'''
        if self.single_feature.name == 'CONSULTATIONS':
            series_durations = series.apply(
                lambda x: [
                    pd.to_datetime(x[i][19:], errors='coerce',dayfirst=True) - \
                    pd.to_datetime(x[i][:16], errors='coerce', dayfirst=True) \
                    for i in range(len(x))
                ])
            return series_durations
        
        else:
        
            series_durations = series.apply(lambda x: 
                                            [
                                             pd.to_datetime(x[i][24:40], dayfirst=True) - \
                                             pd.to_datetime(x[i][:16], dayfirst=True) 
                                             for i in range(len(x))
                                            ])
            return series_durations

In [121]:
# define variables
file = 'emergency_data.xlsx'
visit_date = 'VISIT_DATE'
discharge_date = 'DISCHARGE_DATE'
consultations = 'CONSULTATIONS'
lab_tests = 'LAB_TEST'
radilogy = 'RADIOLOGY'
pharmacy = 'PHARMECY'

## Duration of Leave

In [122]:
# specify class of EmergencyData with three arguments file, visist_date and discharge_date
duration_of_leave = EmergencyData(file, visit_date, discharge_date) 

In [123]:
# compute duration of leave
duration_of_leave.duration_of_leave

0       0 days 02:06:06
1       0 days 01:10:24
2       0 days 01:26:16
3       0 days 00:31:40
4       0 days 00:17:22
              ...      
29951   0 days 05:30:52
29952   0 days 04:42:33
29953   0 days 01:22:24
29954   0 days 05:42:30
29955   0 days 07:00:45
Name: DURATION_OF_LEAVE, Length: 29956, dtype: timedelta64[ns]

In [124]:
# duration of leave "statistics"
duration_of_leave.duration_of_leave_statistics

,DURATION_OF_LEAVE
count,29956
mean,0 days 02:51:16.104920550
std,0 days 15:53:57.306856367
min,0 days 00:00:02
25%,0 days 00:44:47
50%,0 days 01:48:17
75%,0 days 03:33:32.250000
max,68 days 15:03:36


In [125]:
# Percentage of patients who left an emergency department within less or more than and equals four hours
duration_of_leave.percentage_of_duration_specific_hours

,Less than four hours,More than four hours
Percentage,79%,21%


In [126]:
# Percentage of patients who left an emergency department within specific times
duration_of_leave.percentage_of_duration_within_10_20_30_minutes

,within_10_minutes,within_20_minutes,within_30_minutes
Percentage,3%,9%,16%


## Analyzing Consultaions

In [127]:
# specify class of AnalyzingFeature with two arguments consulatioans and duration of leave for inheratencing variables and methods
consultations = AnlyzingFeature(duration_of_leave, consultations)

In [128]:
# consulations feature with null values
consultations.single_feature

0                                                      NaN
1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29953                                                  NaN
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 29956, dtype: object

In [129]:
# consulations feature without null values
consultations.series_without_null_vals

1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
6        Request:02-01-2021 00:40 Reply:02-01-2021 01:2...
7          Request:02-01-2021 00:49 Reply:02-01-2021 01:39
8          Request:02-01-2021 03:24 Reply:02-01-2021 04:06
11         Request:01-01-2021 23:46 Reply:02-01-2021 00:29
                               ...                        
29950      Request:31-03-2021 18:01 Reply:31-03-2021 18:12
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 11508, dtype: object

In [130]:
# consulations feature each order seperated for others
consultations.vals_as_lst

1        [Request:01-01-2021 23:38 Reply:02-01-2021 00:36]
6        [Request:02-01-2021 00:40 Reply:02-01-2021 01:...
7        [Request:02-01-2021 00:49 Reply:02-01-2021 01:39]
8        [Request:02-01-2021 03:24 Reply:02-01-2021 04:06]
11       [Request:01-01-2021 23:46 Reply:02-01-2021 00:29]
                               ...                        
29950    [Request:31-03-2021 18:01 Reply:31-03-2021 18:12]
29951    [Request:31-03-2021 18:58 Reply:01-04-2021 00:37]
29952    [Request:31-03-2021 14:21 Reply:31-03-2021 14:...
29954    [Request:31-03-2021 15:13 Reply:31-03-2021 16:19]
29955    [Request:31-03-2021 14:37 Reply:31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [131]:
# Only dates and times for each order
consultations.lst_dates_times

1                    [01-01-2021 23:38   02-01-2021 00:36]
6        [02-01-2021 00:40   02-01-2021 01:21, 02-01-20...
7                    [02-01-2021 00:49   02-01-2021 01:39]
8                    [02-01-2021 03:24   02-01-2021 04:06]
11                   [01-01-2021 23:46   02-01-2021 00:29]
                               ...                        
29950                [31-03-2021 18:01   31-03-2021 18:12]
29951                [31-03-2021 18:58   01-04-2021 00:37]
29952    [31-03-2021 14:21   31-03-2021 14:30, 31-03-20...
29954                [31-03-2021 15:13   31-03-2021 16:19]
29955                [31-03-2021 14:37   31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [132]:
# lenght of text for each order
consultations.lst_len_orders

1            [47]
6        [47, 47]
7            [47]
8            [47]
11           [47]
           ...   
29950        [47]
29951        [47]
29952    [47, 47]
29954        [47]
29955        [47]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [133]:
# frequents for each consultation count
consultations.order_counts_df

,Frquent
1,9809
2,1242
3,303
4,89
5,35
6,21
7,5
8,3
10,1


In [134]:
# all orders with and without replies"Invalid values"
consultations.unique_no

,"Without Reply/End""Invalid Value",With Reply/End
"Length of Text ""Characters""",31,47
Count of consultations,1208,12720
Percentage of consultations,9%,91%


In [135]:
# compute durations for each order
consultations.duration

1                         [0 days 00:58:00]
6        [0 days 00:41:00, 0 days 00:41:00]
7                         [0 days 00:50:00]
8                         [0 days 00:42:00]
11                        [0 days 00:43:00]
                        ...                
29950                     [0 days 00:11:00]
29951                     [0 days 05:39:00]
29952    [0 days 00:09:00, 0 days 01:02:00]
29954                     [0 days 01:06:00]
29955                     [0 days 04:36:00]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [136]:
# consultations statistics
consultations.duration_stats

,Statistics
count,12720
mean,0 days 01:57:56.316037735
std,0 days 08:25:37.195638597
min,0 days 00:00:00
25%,0 days 00:32:00
50%,0 days 01:02:00
75%,0 days 01:53:00
max,15 days 16:01:00


## Analyzing Lab Tests

In [137]:
lab_tests = AnlyzingFeature(duration_of_leave, lab_tests)

In [128]:
# lab_tests feature with null values
lab_tests.single_feature

0                                                      NaN
1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29953                                                  NaN
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 29956, dtype: object

In [129]:
# lab_tests feature without null values
lab_tests.series_without_null_vals

1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
6        Request:02-01-2021 00:40 Reply:02-01-2021 01:2...
7          Request:02-01-2021 00:49 Reply:02-01-2021 01:39
8          Request:02-01-2021 03:24 Reply:02-01-2021 04:06
11         Request:01-01-2021 23:46 Reply:02-01-2021 00:29
                               ...                        
29950      Request:31-03-2021 18:01 Reply:31-03-2021 18:12
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 11508, dtype: object

In [130]:
# lab_tests feature each order seperated for others
lab_tests.vals_as_lst

1        [Request:01-01-2021 23:38 Reply:02-01-2021 00:36]
6        [Request:02-01-2021 00:40 Reply:02-01-2021 01:...
7        [Request:02-01-2021 00:49 Reply:02-01-2021 01:39]
8        [Request:02-01-2021 03:24 Reply:02-01-2021 04:06]
11       [Request:01-01-2021 23:46 Reply:02-01-2021 00:29]
                               ...                        
29950    [Request:31-03-2021 18:01 Reply:31-03-2021 18:12]
29951    [Request:31-03-2021 18:58 Reply:01-04-2021 00:37]
29952    [Request:31-03-2021 14:21 Reply:31-03-2021 14:...
29954    [Request:31-03-2021 15:13 Reply:31-03-2021 16:19]
29955    [Request:31-03-2021 14:37 Reply:31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [131]:
# Only dates and times for each order
lab_tests.lst_dates_times

1                    [01-01-2021 23:38   02-01-2021 00:36]
6        [02-01-2021 00:40   02-01-2021 01:21, 02-01-20...
7                    [02-01-2021 00:49   02-01-2021 01:39]
8                    [02-01-2021 03:24   02-01-2021 04:06]
11                   [01-01-2021 23:46   02-01-2021 00:29]
                               ...                        
29950                [31-03-2021 18:01   31-03-2021 18:12]
29951                [31-03-2021 18:58   01-04-2021 00:37]
29952    [31-03-2021 14:21   31-03-2021 14:30, 31-03-20...
29954                [31-03-2021 15:13   31-03-2021 16:19]
29955                [31-03-2021 14:37   31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [132]:
# lenght of text for each order
lab_tests.lst_len_orders

1            [47]
6        [47, 47]
7            [47]
8            [47]
11           [47]
           ...   
29950        [47]
29951        [47]
29952    [47, 47]
29954        [47]
29955        [47]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [133]:
# frequents for each lab tests count
lab_tests.order_counts_df

,Frquent
1,9809
2,1242
3,303
4,89
5,35
6,21
7,5
8,3
10,1


In [134]:
# all orders with and without replies"Invalid values"
lab_tests.unique_no

,"Without Reply/End""Invalid Value",With Reply/End
"Length of Text ""Characters""",31,47
Count of consultations,1208,12720
Percentage of consultations,9%,91%


In [135]:
# compute durations for each order
lab_tests.duration

1                         [0 days 00:58:00]
6        [0 days 00:41:00, 0 days 00:41:00]
7                         [0 days 00:50:00]
8                         [0 days 00:42:00]
11                        [0 days 00:43:00]
                        ...                
29950                     [0 days 00:11:00]
29951                     [0 days 05:39:00]
29952    [0 days 00:09:00, 0 days 01:02:00]
29954                     [0 days 01:06:00]
29955                     [0 days 04:36:00]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [136]:
# lab tests statistics
lab_tests.duration_stats

,Statistics
count,12720
mean,0 days 01:57:56.316037735
std,0 days 08:25:37.195638597
min,0 days 00:00:00
25%,0 days 00:32:00
50%,0 days 01:02:00
75%,0 days 01:53:00
max,15 days 16:01:00


## Analyzing Radiolgy

In [147]:
radilogy = AnlyzingFeature(duration_of_leave, radilogy)

In [128]:
# radilogy feature with null values
radilogy.single_feature

0                                                      NaN
1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29953                                                  NaN
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 29956, dtype: object

In [129]:
# radilogy feature without null values
radilogy.series_without_null_vals

1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
6        Request:02-01-2021 00:40 Reply:02-01-2021 01:2...
7          Request:02-01-2021 00:49 Reply:02-01-2021 01:39
8          Request:02-01-2021 03:24 Reply:02-01-2021 04:06
11         Request:01-01-2021 23:46 Reply:02-01-2021 00:29
                               ...                        
29950      Request:31-03-2021 18:01 Reply:31-03-2021 18:12
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 11508, dtype: object

In [130]:
# radilogy feature each order seperated for others
radilogy.vals_as_lst

1        [Request:01-01-2021 23:38 Reply:02-01-2021 00:36]
6        [Request:02-01-2021 00:40 Reply:02-01-2021 01:...
7        [Request:02-01-2021 00:49 Reply:02-01-2021 01:39]
8        [Request:02-01-2021 03:24 Reply:02-01-2021 04:06]
11       [Request:01-01-2021 23:46 Reply:02-01-2021 00:29]
                               ...                        
29950    [Request:31-03-2021 18:01 Reply:31-03-2021 18:12]
29951    [Request:31-03-2021 18:58 Reply:01-04-2021 00:37]
29952    [Request:31-03-2021 14:21 Reply:31-03-2021 14:...
29954    [Request:31-03-2021 15:13 Reply:31-03-2021 16:19]
29955    [Request:31-03-2021 14:37 Reply:31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [131]:
# Only dates and times for each order
radilogy.lst_dates_times

1                    [01-01-2021 23:38   02-01-2021 00:36]
6        [02-01-2021 00:40   02-01-2021 01:21, 02-01-20...
7                    [02-01-2021 00:49   02-01-2021 01:39]
8                    [02-01-2021 03:24   02-01-2021 04:06]
11                   [01-01-2021 23:46   02-01-2021 00:29]
                               ...                        
29950                [31-03-2021 18:01   31-03-2021 18:12]
29951                [31-03-2021 18:58   01-04-2021 00:37]
29952    [31-03-2021 14:21   31-03-2021 14:30, 31-03-20...
29954                [31-03-2021 15:13   31-03-2021 16:19]
29955                [31-03-2021 14:37   31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [132]:
# lenght of text for each order
radilogy.lst_len_orders

1            [47]
6        [47, 47]
7            [47]
8            [47]
11           [47]
           ...   
29950        [47]
29951        [47]
29952    [47, 47]
29954        [47]
29955        [47]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [133]:
# frequents for each radilogy count
radilogy.order_counts_df

,Frquent
1,9809
2,1242
3,303
4,89
5,35
6,21
7,5
8,3
10,1


In [134]:
# all orders with and without replies"Invalid values"
radilogy.unique_no

,"Without Reply/End""Invalid Value",With Reply/End
"Length of Text ""Characters""",31,47
Count of consultations,1208,12720
Percentage of consultations,9%,91%


In [135]:
# compute durations for each order
radilogy.duration

1                         [0 days 00:58:00]
6        [0 days 00:41:00, 0 days 00:41:00]
7                         [0 days 00:50:00]
8                         [0 days 00:42:00]
11                        [0 days 00:43:00]
                        ...                
29950                     [0 days 00:11:00]
29951                     [0 days 05:39:00]
29952    [0 days 00:09:00, 0 days 01:02:00]
29954                     [0 days 01:06:00]
29955                     [0 days 04:36:00]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [136]:
# radilogy statistics
radilogy.duration_stats

,Statistics
count,12720
mean,0 days 01:57:56.316037735
std,0 days 08:25:37.195638597
min,0 days 00:00:00
25%,0 days 00:32:00
50%,0 days 01:02:00
75%,0 days 01:53:00
max,15 days 16:01:00


## Analyzing Pharmacy

In [157]:
pharmacy = AnlyzingFeature(duration_of_leave, pharmacy)

In [128]:
# pharmacy feature with null values
pharmacy.single_feature

0                                                      NaN
1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29953                                                  NaN
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 29956, dtype: object

In [129]:
# pharmacys feature without null values
pharmacy.series_without_null_vals

1          Request:01-01-2021 23:38 Reply:02-01-2021 00:36
6        Request:02-01-2021 00:40 Reply:02-01-2021 01:2...
7          Request:02-01-2021 00:49 Reply:02-01-2021 01:39
8          Request:02-01-2021 03:24 Reply:02-01-2021 04:06
11         Request:01-01-2021 23:46 Reply:02-01-2021 00:29
                               ...                        
29950      Request:31-03-2021 18:01 Reply:31-03-2021 18:12
29951      Request:31-03-2021 18:58 Reply:01-04-2021 00:37
29952    Request:31-03-2021 14:21 Reply:31-03-2021 14:3...
29954      Request:31-03-2021 15:13 Reply:31-03-2021 16:19
29955      Request:31-03-2021 14:37 Reply:31-03-2021 19:13
Name: CONSULTATIONS, Length: 11508, dtype: object

In [130]:
# pharmacys feature each order seperated for others
pharmacy.vals_as_lst

1        [Request:01-01-2021 23:38 Reply:02-01-2021 00:36]
6        [Request:02-01-2021 00:40 Reply:02-01-2021 01:...
7        [Request:02-01-2021 00:49 Reply:02-01-2021 01:39]
8        [Request:02-01-2021 03:24 Reply:02-01-2021 04:06]
11       [Request:01-01-2021 23:46 Reply:02-01-2021 00:29]
                               ...                        
29950    [Request:31-03-2021 18:01 Reply:31-03-2021 18:12]
29951    [Request:31-03-2021 18:58 Reply:01-04-2021 00:37]
29952    [Request:31-03-2021 14:21 Reply:31-03-2021 14:...
29954    [Request:31-03-2021 15:13 Reply:31-03-2021 16:19]
29955    [Request:31-03-2021 14:37 Reply:31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [131]:
# Only dates and times for each order
pharmacy.lst_dates_times

1                    [01-01-2021 23:38   02-01-2021 00:36]
6        [02-01-2021 00:40   02-01-2021 01:21, 02-01-20...
7                    [02-01-2021 00:49   02-01-2021 01:39]
8                    [02-01-2021 03:24   02-01-2021 04:06]
11                   [01-01-2021 23:46   02-01-2021 00:29]
                               ...                        
29950                [31-03-2021 18:01   31-03-2021 18:12]
29951                [31-03-2021 18:58   01-04-2021 00:37]
29952    [31-03-2021 14:21   31-03-2021 14:30, 31-03-20...
29954                [31-03-2021 15:13   31-03-2021 16:19]
29955                [31-03-2021 14:37   31-03-2021 19:13]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [132]:
# lenght of text for each order
pharmacy.lst_len_orders

1            [47]
6        [47, 47]
7            [47]
8            [47]
11           [47]
           ...   
29950        [47]
29951        [47]
29952    [47, 47]
29954        [47]
29955        [47]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [133]:
# frequents for each pharmacy count
pharmacy.order_counts_df

,Frquent
1,9809
2,1242
3,303
4,89
5,35
6,21
7,5
8,3
10,1


In [134]:
# all orders with and without replies"Invalid values"
pharmacy.unique_no

,"Without Reply/End""Invalid Value",With Reply/End
"Length of Text ""Characters""",31,47
Count of consultations,1208,12720
Percentage of consultations,9%,91%


In [135]:
# compute durations for each order
pharmacy.duration

1                         [0 days 00:58:00]
6        [0 days 00:41:00, 0 days 00:41:00]
7                         [0 days 00:50:00]
8                         [0 days 00:42:00]
11                        [0 days 00:43:00]
                        ...                
29950                     [0 days 00:11:00]
29951                     [0 days 05:39:00]
29952    [0 days 00:09:00, 0 days 01:02:00]
29954                     [0 days 01:06:00]
29955                     [0 days 04:36:00]
Name: CONSULTATIONS, Length: 11508, dtype: object

In [136]:
# pharmacy statistics
pharmacy.duration_stats

,Statistics
count,12720
mean,0 days 01:57:56.316037735
std,0 days 08:25:37.195638597
min,0 days 00:00:00
25%,0 days 00:32:00
50%,0 days 01:02:00
75%,0 days 01:53:00
max,15 days 16:01:00
